# CARTE-Enbridge Bootcamp
#### Lab 3

# Predictive Maintenance

In this lab, we are going to take a look at using machine learning to predict when a machine will fail, so you can perform maintenance before it does, or better understand the factors that lead to failure. This is called *predictive maintenance*, and it is a common application of machine learning in industry.

## The Data

The data we are going to use is a synthetic (simulated) dataset modelled after a milling machine used in industrial applications. It was created to assist in measuring model performance for a scientific conference. The original dataset is available [here](https://www.kaggle.com/datasets/stephanmatzka/predictive-maintenance-dataset-ai4i-2020). The data is available as a CSV file, which we will read into a Pandas dataframe:

In [ ]:
import pandas as pd

data = pd.read_csv("https://github.com/alexwolson/carte_workshop_datasets/raw/main/ai4i2020.csv.zip", compression="zip")

In [ ]:
data.head()

As we can see, the data includes a number of features that might be useful in predicting when a machine is likely to fail. The **UDI** column is a unique identifier for each machine, and the **Product ID** column identifies the specific model of machine. The remaining columns (**Air temperature**, **Process temperature**, **Rotational speed**, **Torque**, **Tool wear**, and **Machine failure**) are numeric values that indicate the machine's operating characteristics at a point in time. The **Machine failure** column indicates whether the machine failed in the following hour (1) or not (0). The remaining fields represent different modes of failure:

- TWF: Tool Wear Failure
- HDF: Heat Dissipation Failure
- PWF: Power Failure
- OSF: Overspeed Failure
- RNF: Random Failure

We are going to begin by just predicting whether the machine is going to fail or not, but we can come back to the specific failure modes later.

Let's start by pre-processing our data. We are going to rely on many helpful functions from the [scikit-learn](https://scikit-learn.org/stable/) library, so let's import that now:

In [ ]:
from sklearn import preprocessing

First, we need to split our data into features and labels. The **UDI** and **Product ID** columns are not useful for predicting machine failure, so we can drop them. The **Machine failure** column is our target, so we can drop that from the features and assign it to a separate variable:

In [ ]:
labels = data["Machine failure"]
features = data.drop(
    ["UDI", "Product ID", "Machine failure", "TWF", "HDF", "PWF", "OSF", "RNF"], axis=1
)

The `type` column is a categorical value, so we need to encode it numerically. We can use the `LabelEncoder` class from scikit-learn to do this. Like encoding that we've done before, `LabelEncoder` will assign a unique value to each category, and replace the original value.

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(features["Type"])
features["Type"] = le.transform(features["Type"])

features

It's often helpful to visualize our features to get a sense of what pre-processing we might need to do. We can use the `hist` method of the Pandas dataframe to plot histograms of each feature. It conveniently ignores the categorical features, so we don't have to worry about those, and generates plots for each of the numerical features.

In [ ]:
features.hist(figsize=(12, 12));

The features are all on different scales, so normalizing them will help to ensure that they get considered with equal weight in the model. Scikit-Learn provides a `MinMaxScaler` class that will scale our numeric features to a value between 0 and 1:

In [ ]:
scaler = preprocessing.MinMaxScaler()
features_scaled = scaler.fit_transform(features)
features = pd.DataFrame(features_scaled, columns=features.columns)
features.hist(figsize=(12, 12));

Looking good! Now we have our features and labels prepared, we can split them into training and test sets. We'll use our old friend the `train_test_split` function from scikit-learn to do this. We'll use 20% of the data for testing, and the remaining 80% for training. We'll also set the `random_state` parameter to 0 so that we get the same split every time we run the code. This is useful for reproducibility.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=0
)

print("Training Set: %d, Test Set: %d \n" % (x_train.shape[0], x_test.shape[0]))

## Train a Classification Model

Scikit-Learn includes a variety of machine learning algorithms that we can use to train a classification model. Let's try a simple logistic regression model first. We are going to measure the time it takes each of our models to train today, so we can compare them later.

In [ ]:
from sklearn.linear_model import LogisticRegression
from time import time

lr = LogisticRegression()
start_time = time()
lr.fit(x_train, y_train)
print(f'Training time: {time() - start_time:.2f}s')

The way that we measure the performance of a model is dependent on the task that we are performing. When it comes to predictive maintenance, it's probably more important that we don't miss a machine failure than it is to avoid false alarms. In other words, we want to minimize the number of *false negatives* (predicting that the machine will not fail in the next hour when it actually does) rather than the number of *false positives* (predicting that the machine will fail when it actually doesn't). 

We can calculate these metrics directly:

In [ ]:
import numpy as np


def get_negatives_metrics(x, y, model):
    y_hat = model.predict(x)
    false_negatives = np.logical_and((y_hat == 0), (y == 1)).sum()
    false_positives = np.logical_and((y_hat == 1), (y == 0)).sum()
    return false_negatives, false_positives

In [ ]:
false_negatives, false_positives = get_negatives_metrics(x_test, y_test, lr)
print(f"False Negatives: {false_negatives}, False Positives: {false_positives}")

As we can see, the Logistic Regression model has way too many false negatives to be useful for predictive maintenance. Another way to look at this is using a balanced accuracy score. While the standard accuracy metric is the number of correct predictions divided by the total number of predictions, the balanced accuracy score is the average of the recall obtained on each class. In other words, it's the average of the true positive rate and the true negative rate. We can calculate this using the `balanced_accuracy_score` function from scikit-learn:

In [ ]:
from sklearn.metrics import balanced_accuracy_score

print(
    f"Balanced accuracy: {balanced_accuracy_score(y_test, lr.predict(x_test)) * 100:.2f}%"
)

50% accuracy for a binary task means that our model is no better than a coin flip. Clearly, this Logistic Regression model is not going to be useful for predictive maintenance. Let's try a more sophisticated algorithm - a decision tree. In order to help with visualization, we're going to limit the depth of the tree to 3 - this means that the tree will only ask 3 questions about the data before making a prediction. We'll also measure the training time again.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=3)
start_time = time()
dt.fit(x_train, y_train)
print(f'Training time: {time() - start_time:.2f}s')

false_negatives, false_positives = get_negatives_metrics(x_test, y_test, dt)
print(f"False Negatives: {false_negatives}, False Positives: {false_positives}")
print(
    f"Balanced accuracy: {balanced_accuracy_score(y_test, dt.predict(x_test)) * 100:.2f}%"
)

We're getting better! While the number of false positives did increase, the number of false negatives _decreased_ dramatically. Since that's what we care about, this is a big improvement.
 
One good thing about decision trees is that it's easy to follow the logic that the model is using to make predictions. We can visualize the decision tree using the `plot_tree` function from scikit-learn:

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree

plt.figure(figsize=(20, 10))
tree.plot_tree(
    dt,
    feature_names=features.columns,
    class_names=["No Failure", "Failure"],
    filled=True,
    rounded=True,
);

The tree plot shows us how the model is making predictions. Each node represents a "question" being asked about the data. For instance, the first node is `Torque[Nm] <= 0.785`. For a given sample, if the measured torque is less than or equal to 0.785Nm, we follow the left branch, otherwise we follow the right branch. 

The `gini` value is a measure that indicates how many samples in each node are in a single class. A `gini` value of 0 means that all of the samples in the node are in the same class, while a value of 0.5 means that the samples are evenly split between the two classes. This is actually the same gini coefficient that is used to measure income inequality in economics.

**Your Turn**

Try changing the `max_depth` parameter of the decision tree model to see if you can improve the balanced accuracy score.

---

Let's try one more algorithm: a random forest. A random forest is an ensemble of decision trees, which means that it combines the predictions of multiple decision trees to improve the overall performance. It's typically more accurate than a single decision tree, but it's harder to interpret, since we can't visualize the entire forest.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
start_time = time()
rf.fit(x_train, y_train)
print(f'Training time: {time() - start_time:.2f}s')

false_negatives, false_positives = get_negatives_metrics(x_test, y_test, rf)
print(f"False Negatives: {false_negatives}, False Positives: {false_positives}")
print(
    f"Balanced accuracy: {balanced_accuracy_score(y_test, rf.predict(x_test)) * 100:.2f}%"
)

**Your Turn**

Is this better or worse than the decision tree? How do we decide?

## Interpreting the Model

One of the challenges with machine learning is that it can be difficult to understand how the model is making predictions. This is especially true for more complex algorithms, such as random forests. 

One way to get a sense of how the model is making predictions is to look at the feature importances. The feature importances indicate how much each feature contributes to the overall prediction. 

We can get the feature importances from the random forest model:

In [ ]:
feature_importances = pd.DataFrame(
    rf.feature_importances_, index=x_train.columns, columns=["importance"]
).sort_values("importance", ascending=False)
feature_importances.plot.bar();

As we can see, the `Torque [Nm]` feature is the most important in the model. Least important is the `Type` feature. Investigating feature importance is useful to understand which measurements are worth taking, and which might not be.

Another way to understand how the model is making predictions is to look at the partial dependence of the model on each feature. The partial dependence is the marginal effect of a feature on the predicted outcome. In other words, it shows us how the probability of failure changes as we change the value of a feature.

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

fig, ax = plt.subplots(figsize=(12, 8))
disp = PartialDependenceDisplay.from_estimator(rf, x_train, ["Torque [Nm]"], ax=ax)

This plot shows us that as the torque increases, the probability of failure increases. Go ahead and try plotting the partial dependence for some of the other features.

## Predicting Failure Modes

So far, we have only been predicting whether the machine will fail or not. Let's see if we can predict the specific failure mode. We are going to use the random forest model that we trained above, but we are going to add the failure mode columns back into the features:

In [ ]:
labels = data[["TWF", "HDF", "PWF", "OSF", "RNF"]]
features = data.drop(
    ["UDI", "Product ID", "Machine failure", "TWF", "HDF", "PWF", "OSF", "RNF"], axis=1
)

# Encode the Type column
features["Type"] = le.transform(features["Type"])
features_scaled = scaler.fit_transform(features)
features = pd.DataFrame(features_scaled, columns=features.columns)

# Split the data into training and test sets
x_train_modes, x_test_modes, y_train_modes, y_test_modes = train_test_split(
    features, labels, test_size=0.2, random_state=0
)

In [ ]:
rf = RandomForestClassifier()
start_time = time()
rf.fit(x_train_modes, y_train_modes)
print(f'Training time: {time() - start_time:.2f}s')

Since we now have 5 different failure modes, we need to use a different metric to evaluate the model. We can use the balanced accuracy score for each failure mode, and then average them together to get an overall score:

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

y_hat = rf.predict(x_test_modes)
accuracies = []
for i, confusion in enumerate(multilabel_confusion_matrix(y_test_modes, y_hat)):
    print(labels.columns[i])
    print(f"False Negatives: {confusion[1, 0]}")
    print(f"False Positives: {confusion[0, 1]}")
    balanced_accuracy = balanced_accuracy_score(y_test_modes.iloc[:, i], y_hat[:, i])
    accuracies.append(balanced_accuracy)
    print(
        f"Balanced accuracy: {balanced_accuracy * 100:.2f}%"
    )
    print()
    
print(f"Average balanced accuracy: {np.mean(accuracies) * 100:.2f}%")

Looks like we are doing well at predicting the `HDF`, `PWF` and `OSF` failure modes, but not so well at predicting the `TWF` and `RNF` failure modes.

**Your Turn**

Try changing the parameters of the random forest model to see if you can improve the balanced accuracy score for the `TWF` and `RNF` failure modes. If you'd like to go further, you could also try using a different ensemble method. For documentation on the ensemble methods available in scikit-learn, see [here](https://scikit-learn.org/stable/modules/ensemble.html).

In [ ]:
# Your code here

# Bonus: Testing Explainable Boosting Machines

As we discussed in the lecture, Explainable Boosting Machines (EBMs) are a relatively new machine learning algorithm that is designed to be more interpretable than other algorithms. Let's see if we can use an EBM to predict machine failure. We are going to use the [interpret](https://interpret.ml/) library from Microsoft to train the EBM, then we can compare its performance to the random forest model that we trained above.

In [ ]:
!pip install -q -U interpret

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier

ebm = ExplainableBoostingClassifier()
start_time = time()
ebm.fit(x_train, y_train)
print(f'Training time: {time() - start_time:.2f}s')

In [ ]:
false_negatives, false_positives = get_negatives_metrics(x_test, y_test, ebm)
print(f"False Negatives: {false_negatives}, False Positives: {false_positives}")

One great feature of Microsoft's interpret library is that it includes a dashboard that we can use to visualize the model's performance. We can use the `show` method of the `ExplanationDashboard` class to launch the dashboard:

In [ ]:
from interpret import show

ebm_global = ebm.explain_global()

show(ebm_global)